In [2]:
import cv2
from ultralytics import YOLOWorld
import supervision as sv

In [35]:
detections.xyxy

array([      890.8], dtype=float32)

In [33]:
image = cv2.imread("25612.jpg")
model = YOLOWorld("yolov8x-worldv2.pt")

# Define custom classes
model.set_classes(["license_plate", ""])

# Execute prediction for specified categories on an image
results = model.predict(image, conf=0.1)

# Получение детекций для Supervision
detections = sv.Detections.from_ultralytics(results[0])
# фильтрация детекций по ширине bounding-box-а
w = detections.xyxy[:, 2] - detections.xyxy[:, 0]
h = detections.xyxy[:, 3] - detections.xyxy[:, 1]
detections = detections[(w < 500)]

# Визуализация боксов и подписей
annotator = sv.BoxAnnotator()
label_annotator = sv.LabelAnnotator()

# вывод доп информации (уверенность)
labels = [
    f"{class_name} {confidence:.2f}"
    for class_name, confidence
    in zip(detections['class_name'], detections.confidence)
]
annotated_image = annotator.annotate(scene=image, detections=detections)
annotated_image = label_annotator.annotate(scene=annotated_image, detections=detections, labels=labels)

cv2.imwrite("annotated.jpg", annotated_image)


0: 480x640 2 license_plates, 138.8ms
Speed: 2.0ms preprocess, 138.8ms inference, 3.7ms postprocess per image at shape (1, 3, 480, 640)


True